- 如果在 Transformer Encoder 中没有使用位置编码，那么模型将无法区分输入序列中各个词的顺序，这实际上等同于一个词袋（Bag of Words）模型。原因是 Transformer 的自注意力机制本质上是对输入的加权求和，而没有位置编码的情况下，模型无法获取任何位置信息。
- Permutation Equivariance（排列等变）
    - Permutation Equivariance（排列等变）：如果对输入序列进行某种排列，模型的输出将以相同的方式被排列。
    - Permutation Invariance（排列不变）：对输入序列的排列不会影响模型的输出，即输出与输入的排列无关。
    - 没有位置编码的 Transformer Encoder 并不是排列不变的，而是排列等变的。这意味着如果我们改变输入序列中词的顺序，输出序列中的元素也会按照相同的方式重新排列，但输出本身的数值不会保持不变。

In [1]:
import torch
import torch.nn as nn

In [2]:
torch.manual_seed(42)

In [3]:
# 定义模型参数
vocab_size = 10000  # 词汇表大小
d_model = 512       # 嵌入维度
nhead = 8           # 注意力头数
num_layers = 1      # Transformer Encoder 层数

### w/o pe

In [4]:
# 定义嵌入层和 Transformer Encoder
embedding = nn.Embedding(vocab_size, d_model)
# dropout == 0.
encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=0.0)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

/home/whaow/anaconda3/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [5]:
# 生成随机输入序列
seq_len = 10  # 序列长度
input_ids = torch.randint(0, vocab_size, (seq_len,))

# 打乱输入序列
perm = torch.randperm(seq_len)
shuffled_input_ids = input_ids[perm]

In [6]:
perm, torch.argsort(perm)

(tensor([2, 4, 7, 3, 0, 5, 8, 9, 6, 1]),
 tensor([4, 9, 0, 3, 1, 5, 8, 2, 6, 7]))

In [7]:
# 获取嵌入表示
embedded_input = embedding(input_ids)  # [seq_len, d_model]
embedded_shuffled_input = embedding(shuffled_input_ids)

In [8]:
# Transformer 期望的输入形状为 [seq_len, batch_size, d_model]，因此需要调整维度。

# 添加 batch 维度
# [seq_len, 1, d_model]
embedded_input = embedded_input.unsqueeze(1)           
embedded_shuffled_input = embedded_shuffled_input.unsqueeze(1)

# 通过 Transformer Encoder
output = transformer_encoder(embedded_input)           # [seq_len, 1, d_model]
output_shuffled = transformer_encoder(embedded_shuffled_input)

In [9]:
output.shape, output_shuffled.shape

(torch.Size([10, 1, 512]), torch.Size([10, 1, 512]))

In [10]:
torch.allclose(output, output_shuffled, atol=1e-6)

False

In [11]:
are_outputs_equal = torch.allclose(output.squeeze(1).mean(dim=0), output_shuffled.squeeze(1).mean(dim=0), atol=1e-6)
are_outputs_equal

True

In [12]:
inverse_perm = torch.argsort(perm)
output_shuffled_reordered = output_shuffled[inverse_perm]

In [13]:
torch.allclose(output, output_shuffled_reordered, atol=1e-6)

True

### with pe

In [16]:
# 定义位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        # 创建位置编码矩阵
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.pe = pe

    def forward(self, x):
        # 加性位置编码
        x = x + self.pe[:x.size(0)]
        return x

In [15]:
# 添加位置编码
pos_encoder = PositionalEncoding(d_model)
embedded_input_pos = pos_encoder(embedded_input)
embedded_shuffled_input_pos = pos_encoder(embedded_shuffled_input)

In [17]:
# 通过 Transformer Encoder
output_pe = transformer_encoder(embedded_input_pos)
output_shuffled_pe = transformer_encoder(embedded_shuffled_input_pos)

In [18]:
output_pe.shape, output_shuffled_pe.shape

(torch.Size([10, 1, 512]), torch.Size([10, 1, 512]))

In [19]:
torch.allclose(output_pe.squeeze(1).mean(dim=0), output_shuffled_pe.squeeze(1).mean(dim=0), atol=1e-6)

False